# 1th_homework

### Import Section

In [134]:
import numpy as np
import pandas as pd

import torch

import math

## 1. Hit Rate@k

**Исходные данные:**
- recommended_list - список id рекомендованных товаров;
- bought_list - список id купленных товаров.

In [1]:
recommended_list = [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43]
bought_list = [521, 32, 143, 991]

**Исходная метрика Hit Rate**

In [5]:
def hit_rate(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    hit_rate = (flags.sum() > 0) * 1
    
    return hit_rate

In [6]:
hit_rate(recommended_list, bought_list)

1

**Усовершенствованная метрика Hit Rate@k**

Предполагается, что id рекомендованных товаров расположены в списке recommended_list в порядке убывания критерия рекомендации, то есть сперва идут самые рекомендованные. В теле функции происходит проверка не по всем товарам, а только по срезу из k первых рекомендованных товаров.

In [72]:
def hit_rate_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list[:k])
    
    flags = np.isin(bought_list, recommended_list)
    
    hit_rate = int(flags.sum() > 0)
    
    return hit_rate

In [73]:
hit_rate_at_k(recommended_list, bought_list)

1

In [74]:
hit_rate_at_k(recommended_list, bought_list[:2], k=3)

0

## 2. Money Precision@k

**Сгенерируем цены для товаров.**

Цены будут храниться в словаре с доступом на уникальному id товара, то есть соответствие {id: цена}.

In [21]:
# Набор всех уникальных id.
set_all_ids = set(recommended_list + bought_list)

# Генерация словаря со случайными ценами для каждого id товара от 0 до 1000.
dict_prices = dict(zip(set_all_ids, np.random.randint(0, 1000, len(set_all_ids))))

dict_prices

{32: 428,
 1543: 586,
 521: 373,
 11: 919,
 43: 272,
 1134: 26,
 143: 893,
 3345: 764,
 533: 377,
 27: 461,
 156: 833,
 991: 305}

**Метрика Money Precision@k**

Метрика вычисляет долю суммы купленных рекомендованных товаров от суммы всех рекомендованных товаров.

In [75]:
def money_precision_at_k(recommended_list, bought_list, prices_recommended, k=5):
    
    array_recommended = np.array(recommended_list[:k])
    array_bought = np.array(bought_list)
    array_prices_recommended = np.array(prices_recommended[:k])
    
    # Массив-маска флагов, 1 - товар был куплен, 0 - товар не был куплен.
    array_flags = (np.isin(array_recommended, array_bought)).astype(int)
    
    # Вычисление точности: отношение сумм первых k купленных рекомендованных товаров и k рекомендованных товаров.
    precision = (array_prices_recommended @ array_flags) / array_prices_recommended.sum()
    
    return np.round(precision, 3)

**Список цен рекомендованных товаров**

In [45]:
prices_recommended = list(map(dict_prices.get, recommended_list))

prices_recommended

[893, 833, 26, 305, 461, 586, 764, 377, 919, 272]

**Проверка работы функции**

In [71]:
money_precision_at_k(recommended_list, bought_list, prices_recommended, k=3)

0.51

In [63]:
print(f'''
Флаг, куплен ли товар:\t\t\t{list((np.isin(recommended_list, bought_list)).astype(int))}
Стоимость рекомендованного товара:\t{prices_recommended}
''')


Флаг, куплен ли товар:			[1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
Стоимость рекомендованного товара:	[893, 833, 26, 305, 461, 586, 764, 377, 919, 272]



Метрика должна была оценить долю стоимостей первых трёх рекомендованных товаров.

    Первый товар: стоимость 893, был куплен.
    Второй товар: стоимость 833, не был куплен.
    Третий товар: стоимость 26,  не был куплен.

$$a = \frac{893 * 1 + 833 * 0 + 26 * 0}{893 * 1 + 833 * 1 + 26 * 1} = \frac{893}{1752} \approx 0.51$$

Значения совпадают, метрика работает корректно.

## 3. Money Recall@k

**Метрика Money Recall@k**

Метрика вычисляет долю суммы купленных рекомендованных товаров от суммы всех купленных товаров.

In [90]:
def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):
    
    array_recommended = np.array(recommended_list[:k])
    array_bought = np.array(bought_list)
    array_prices_recommended = np.array(prices_recommended[:k])
    array_prices_bought = np.array(prices_bought)
    
    # Массив-маска флагов, 1 - товар был куплен, 0 - товар не был куплен.
    array_flags = (np.isin(array_recommended, array_bought)).astype(int)
    
    # Вычисление полноты: отношение сумм первых k купленных рекомендованных товаров и всех купленных товаров.
    recall = (array_prices_recommended @ array_flags) / array_prices_bought.sum()
    
    return np.round(recall, 3)

**Список цен купленных товаров**

In [80]:
prices_bought = list(map(dict_prices.get, bought_list))

prices_bought

[373, 428, 893, 305]

**Проверка работы функции**

In [91]:
money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=3)

0.447

In [92]:
print(f'''
Флаг, куплен ли товар:\t\t\t{list((np.isin(recommended_list, bought_list)).astype(int))}
Стоимость рекомендованного товара:\t{prices_recommended}
Стоимость купленного товара:\t\t{prices_bought}
''')


Флаг, куплен ли товар:			[1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
Стоимость рекомендованного товара:	[893, 833, 26, 305, 461, 586, 764, 377, 919, 272]
Стоимость купленного товара:		[373, 428, 893, 305]



Метрика должна была оценить долю стоимостей первых трёх рекомендованных товаров.

    Первый рекомендованный товар: стоимость 893, был куплен.
    Второй рекомендованный товар: стоимость 833, не был куплен.
    Третий рекомендованный товар: стоимость 26,  не был куплен.
    
    Первый купленный товар: стоимость 373.
    Второй купленный товар: стоимость 428.
    Третий купленный товар: стоимость 893.
    Четвёртый купленный товар: стоимость 305.

$$a = \frac{893 * 1 + 833 * 0 + 26 * 0}{373 + 428 + 893 + 305} = \frac{893}{1999} \approx 0.447$$

Значения совпадают, метрика работает корректно.

## 4. Mean Average Precision@k

**Исходная метрика Average Precision@k**

In [95]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    #assert len(bought_list) > len(recommended_list)
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    return precision


def ap_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(recommended_list, bought_list)
    
    if sum(flags) == 0:
        return 0
    
    sum_ = 0
    for i in range(0, k-1):
        if flags[i] == True:
            p_k = precision_at_k(recommended_list, bought_list, k=i+1)
            sum_ += p_k
            
    result = sum_ / sum(flags)
    
    return result

In [96]:
ap_k(recommended_list, bought_list, k=5)

0.75

**Исходные данные:**
- recommended_list - список списков id рекомендованных товаров, для каждого клиента отдельный список;
- bought_list - список списков id купленных товаров, для каждого клиента отдельный список.

In [97]:
recommended_list = [[143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43],
                    [146, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43],]

bought_list = [[521, 32, 143, 991],
               [146, 29]]

**Метрика Mean Average Precision@k**

Данная метрика, аналогично Average Precision@k, вычисляет среднюю точность для первых k покупок. Однако Mean Average Precision@k вычисляет среднюю точность для нескольких пользователей, затем возвращает средневзвешенную точность по всем клиентам.

In [123]:
def map_k(recommended_list, bought_list, k=5, u=1):
    
    # Защита от ввода некорректного аргумента u.
    len_list = len(recommended_list)
    if (u < 0) or (u > len_list) or type(u) != int:
        u = len_list
    
    result = 0
    
    # Для каждого клиента:
    for i in range(u):
        # вычислить среднюю точность первых k покупок.
        result += ap_k(recommended_list[i], bought_list[i], k)
    
    # Усреднение метрики по всем клиентам.
    result /= u
    
    return result

In [124]:
map_k(recommended_list, bought_list, k=3, u=1)

0.5

In [125]:
map_k(recommended_list, bought_list, k=3, u=2)

0.75

## 5. Normalized Discounted Cumulative Gain@k

In [128]:
N = 5
ys_true = torch.randint(0, 5, (N, ))
ys_pred = torch.rand(N)

Истинные метки, чем больше число, тем выше заинтересованность клиента в объекте.

In [129]:
ys_true

tensor([4, 1, 0, 2, 3])

Результат прогноза модели - коэффициент важности объекта. Чем больше число, тем приоритетнее объект.

In [131]:
ys_pred

tensor([0.2074, 0.7182, 0.4096, 0.0619, 0.8712])

Ранжирование объектов согласно прогнозу модели: на первом месте объект с индексом 4 и т.д.

In [133]:
_, indices = torch.sort(ys_pred, descending=True, dim=0)

indices

tensor([4, 1, 2, 0, 3])

**Метрика Discounted cumulative gain@k**

In [137]:
def compute_gain(y_value: float, gain_scheme: str) -> float:
    if gain_scheme == "exp2":
        gain = 2 ** y_value - 1
    elif gain_scheme == "const":
        gain = y_value
    else:
        raise ValueError(f"{gain_scheme} method not supported, only exp2 and const.")
    return float(gain)


def dcg(ys_true: torch.Tensor, ys_pred: torch.Tensor, gain_scheme: str) -> float:
    _, argsort = torch.sort(ys_pred, descending=True, dim=0)
    ys_true_sorted = ys_true[argsort]
    ret = 0
    for idx, cur_y in enumerate(ys_true_sorted, 1):
        gain = compute_gain(cur_y, gain_scheme)
        
        # your code
        
        # Для каждого элемента после второго элемента:
        if idx > 2:
            # штрафовать прирост в зависимости от расположения.
            gain /= math.log2(idx)
        
        # Суммирование прироста.
        ret += gain
    
    # Нормирование по количеству объектов в пронозе.
    ret /= len(ys_true)
        
    return ret

In [139]:
dcg(ys_true, ys_pred, 'const')

1.3722706232293571

Попробуем перегенерировать исходные данные, чтобы получить другое значение метрики.

In [140]:
ys_true = torch.randint(0, 5, (N, ))
ys_pred = torch.rand(N)

ys_true, ys_pred

(tensor([3, 3, 1, 2, 1]), tensor([0.1212, 0.7432, 0.1412, 0.1689, 0.1709]))

In [141]:
dcg(ys_true, ys_pred, 'const')

1.410777836272619

Значение метрики выше, так как один из важных объектов был поставлен моделью на первое место.

## Mean Reciprocal Rank@k

In [142]:
recommended_list = [[143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43],
                    [146, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43],]

bought_list = [[521, 32, 143, 991],
               [146, 29]]

В контексте покупки товара метрика будет учитывать позицию купленного товара в писке рекомендованных товаров. Если купленного товара нет в списке рекомендованных, то его обратный ранг будет равен нулю, что снизит среднее по всем рангам.

In [153]:
def reciprocal_rank(recommended_list, bought_list):
    
    result = 0
    
    # Для каждого купленного товара:
    for item in bought_list:
        # пробовать найти его позицию в списке рекомендаций.
        try:
            # Если товар найден, то вычислить его обратный ранг.
            rank = 1 / (1 + recommended_list.index(item))
        except:
            # Иначе использовать нулевой ранг.
            rank = 0
        
        # Суммировать ранги.
        result += rank
    
    # Вычислить среднее.
    result /= len(bought_list)
    
    return result

In [154]:
reciprocal_rank(recommended_list[0], bought_list[0])

0.3125

У первого клиента из четёрых рекомендованных товаров только два оказались куплены. Первый товар был на первом месте, второй - на четвёртом.

In [155]:
reciprocal_rank(recommended_list[1], bought_list[1])

0.5

У второго клиента был верно предсказан только один товар из двух, однако предсказанный товар был на первом месте, поэтому метрика получилась выше. 

In [156]:
recommended_list = [[1, 2, 3, 4],
                    [1, 2, 3, 4]]

bought_list = [[1, 2, 3, 4],
               [1]]

In [157]:
reciprocal_rank(recommended_list[0], bought_list[0])

0.5208333333333333

In [158]:
reciprocal_rank(recommended_list[1], bought_list[1])

1.0

Данная метрика необъективна в случае нескольких покупок из одного списка рекомендаций. Для первого клиента модель сделала верную рекомендацию для всех его покупок, однако метрика получилась ниже, чем для второго клиента с одной покупкой.